In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Подготовка данных

In [0]:
train = pd.read_csv('https://raw.githubusercontent.com/SESCDS/sescds/master/Lesson3/train.csv')
test = pd.read_csv("https://raw.githubusercontent.com/SESCDS/sescds/master/Lesson3/test.csv")

In [0]:
train['is_test'] = False
test['is_test'] = True
test['Survived'] = True
data = pd.concat([train, test])

In [0]:
data['Title'] = data['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
title = {'Mr':'Mr', 'Mrs':'Mrs', 'Miss':'Miss', 'Master':'Master', 'Don':'Royal', 'Rev':'Royal', 'Dr':'Dr', 'Mme':'Royal',
        'Ms':'Miss', 'Major':'Royal', 'Lady':'Royal', 'Sir':'Royal', 'Mlle':'Miss', 'Col':'Royal', 'Capt':'Royal',
        'the Countess':'Royal', 'Jonkheer':'Royal', 'Dona':'Royal'}
data['Title'] = data['Title'].map(title)

In [0]:
cat_vars = ['Cabin', 'Embarked', 'Title', 'Pclass', 'Sex']
for var in cat_vars: data[var] = data[var].astype('category').cat.codes

In [0]:
data['Age'] = data['Age'].fillna(data['Age'].median())

In [0]:
data['Fare'] = data['Fare'].fillna(data['Fare'].median())

In [0]:
train = data[~data['is_test']]
test = data[data['is_test']]

In [0]:
drop_columns = ['Name', 'Ticket', 'is_test','Survived']
features = data.columns.difference(drop_columns)

# Разбиение на тренировочную/тестовую выборки

Как вы помните, при построении модели у нас есть две основных проблемы: недообучение и переобучение (*что это?*). Чтобы их избежать, нужно оценить модель. Самы простой способ для этого - разбить данные на трейн и тест, обучить модель на трейне и проверить на тесте. Раньше вы загружали предсказания по тесту, чтобы получить результат, но можно воспользоваться тем, что есть на руках :)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [0]:
X = train[features]
y = train['Survived']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(623, 10) (623,)
(268, 10) (268,)


In [14]:
model_tree = DecisionTreeClassifier()
model_tree.fit(X_train, y_train)
print('Train acc:', model_tree.score(X_train, y_train))
print('Test acc:', model_tree.score(X_test, y_test))

Train acc: 1.0
Test acc: 0.75


*Результат хуже, чем на прошлом уроке. Почему?*
*Как зависит результат от параметра test_size? Почему?*

# Подбор параметров

Посмотрим на модель. У нее много параметров, а мы оставили все по умолчанию. Попробуйте изменять их. Посмотрим, удастся ли улучшить результат.

In [15]:
model_tree

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [16]:
model_tree = DecisionTreeClassifier()
model_tree.fit(X_train, y_train)
print('Train acc:', model_tree.score(X_train, y_train))
print('Test acc:', model_tree.score(X_test, y_test))

Train acc: 1.0
Test acc: 0.7611940298507462


# Кросс-валидация

Логичное развитие метода разбиения на трейн и тест - повторить его несколько раз и усреднить результаты!

<img src="http://www.feiguyunai.com/wp-content/uploads/2018/09/ec9f62166bc64ced29f84034f5556159.png">

In [0]:
from sklearn.model_selection import cross_val_score

In [0]:
from sklearn.model_selection import ShuffleSplit

In [20]:
cv = ShuffleSplit(n_splits=5, random_state=None)
cross_val_score(model_tree, X, y, cv=5)

array([0.62569832, 0.75977654, 0.79213483, 0.80337079, 0.7740113 ])

В данном случае выборка не перемешивается, и мы видим, как различаются результаты в зависимости от расположения в данных.
Попробуем перемешать

In [28]:
cv = ShuffleSplit(n_splits=5, random_state=0)
cross_val_score(model_tree, X, y, cv=cv)

array([0.7       , 0.68888889, 0.81111111, 0.78888889, 0.73333333])

Стало чуть ровнее, но разброс всё еще большой. Результаты предсказаний сильно зависят от разбиения (*почему?*)

Посмотрите на результаты кросс-валидации для вашей модели

# Ансамбли

А что если попробовать обучить несколько деревьев на части данных, а потом результаты усреднить?

В цикле обучайте модель на части наблюдений, делайте предсказания и сохраняйте результаты в массив. В другой массив сохраняйте метрику (сделайте два массива - трейн и тест). После завершения цикла возьмите среднее от предсказаний всех деревьев и посчитайте для него метрику. Сравните со средней метрикой по одиночным деревьям.

In [29]:
# так можно делать выборку
X_train_tmp = X_train.sample(frac=0.7)
model_tree.fit(X_train_tmp, y_train[X_train_tmp.index])
pr = model_tree.predict(X_test)

accuracy_score(pr, y_test)

0.7238805970149254

То, что мы сейчас сделали, называется случайным лесом - это уже очень сильный алгоритм, который хорошо защищен от главной проблемы деревьев - переобучения. Домашнее задание - изучите RandomForestClassifier, подберите параметры с помощью кросс-валидации и сделайте сабмит на Kaggle.